In [1]:
import sys
from scipy.sparse import load_npz, save_npz
sys.path.insert(0, '../src/')

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import scipy.sparse as sp
import torch

import utils
from net import start_experiments
from baselines import (Forge_Adjacency, Forge_Modularity, Forge_SymmetricLaplacian, Forge_Transition,
                       train_and_log_multiple_conf_models, Transition)

In [2]:
_A_obs = load_npz('../data/CORA_ML.npz')

In [3]:
val_share = 0.1
test_share = 0.05
seed = 481516234

train_ones, val_ones, val_zeros, test_ones, test_zeros = utils.train_val_test_split_adjacency(_A_obs, val_share, test_share, seed, undirected=True, connected=True, asserts=False)

train_graph = sp.csr_matrix((np.ones(len(train_ones)),(train_ones[:,0], train_ones[:,1])))
assert (train_graph.toarray() == train_graph.toarray().T).all()
val_edges = (val_ones, val_zeros)

## Forge Framework

In [4]:
fa = Forge_Adjacency(A=train_graph, rank=1600)
graphs_fa = fa(sample_size=20,
               logdir='../logs/CORA-ML/baseline_FA',
               val_edges=val_edges)

In [5]:
ft = Forge_Transition(A=train_graph, rank=1600)
graphs_ft = ft(sample_size=20,
               logdir='../logs/CORA-ML/baseline_FT',
               val_edges=val_edges)

In [6]:
fm = Forge_Modularity(A=train_graph, rank=1600)
graphs_fm = fm(sample_size=20,
               logdir='../logs/CORA-ML/baseline_FM',
               val_edges=val_edges)

In [15]:
fs = Forge_SymmetricLaplacian(A=train_graph, rank=2530)
graphs_fs = fs(sample_size=20,
               logdir='../logs/CORA-ML/baseline_FS',
               val_edges=val_edges)

## Transition with Cross Entropy

In [5]:
models = start_experiments(num_experiments=5,
                           experiment_root='../logs/CORA-ML/baseline_transition_ce_loss',
                           train_graph=train_graph,
                           H=950,
                           optimizer=torch.optim.Adam,
                           optimizer_args={'lr': 0.1,
                                           'weight_decay': 1e-7},
                           invoke_every=5,
                           steps = 100,
                           Model=Transition,
                           val_edges=(val_ones, val_zeros))


Experiment_0
Step:   5/100, Loss: 7.72731, Edge-Overlap: 0.002
Step:  10/100, Loss: 7.57919, Edge-Overlap: 0.004
Step:  15/100, Loss: 7.44946, Edge-Overlap: 0.006
Step:  20/100, Loss: 7.23589, Edge-Overlap: 0.008
Step:  25/100, Loss: 6.92385, Edge-Overlap: 0.011
Step:  30/100, Loss: 6.56970, Edge-Overlap: 0.017
Step:  35/100, Loss: 6.21148, Edge-Overlap: 0.020
Step:  40/100, Loss: 5.85016, Edge-Overlap: 0.032
Step:  45/100, Loss: 5.47775, Edge-Overlap: 0.046
Step:  50/100, Loss: 5.09422, Edge-Overlap: 0.069
Step:  55/100, Loss: 4.70832, Edge-Overlap: 0.097
Step:  60/100, Loss: 4.32631, Edge-Overlap: 0.128
Step:  65/100, Loss: 3.95522, Edge-Overlap: 0.192
Step:  70/100, Loss: 3.61128, Edge-Overlap: 0.245
Step:  75/100, Loss: 3.31622, Edge-Overlap: 0.316
Step:  80/100, Loss: 3.09036, Edge-Overlap: 0.397
Step:  85/100, Loss: 2.93726, Edge-Overlap: 0.448
Step:  90/100, Loss: 2.84310, Edge-Overlap: 0.481
Step:  95/100, Loss: 2.78743, Edge-Overlap: 0.501
Step: 100/100, Loss: 2.75410, Edge-O

## Configuration Model

In [9]:
train_and_log_multiple_conf_models(logdir='../logs/CORA-ML/baseline_conf/',
                                   A=train_graph,
                                   EOs=[0.3, 0.5, 0.7],
                                   experiments_per_EO=20)